In [1]:
# Imports

import glob
from eqcorrscan.core.match_filter import Template
import os
from eqcorrscan import tests

In [2]:
# Template Creation

# Get the path for the test-data so we can test this
TEST_PATH = os.path.dirname(tests.__file__)
sac_files = glob.glob(TEST_PATH + '/test_data/SAC/2014p611252/*')
# sac_files is now a list of all the SAC files for event id:2014p611252
template = Template().construct(
    method='from_sac', name='test', lowcut=2.0, highcut=8.0,
    samp_rate=20.0, filt_order=4, prepick=0.1, swin='all',
    length=2.0, sac_files=sac_files)

In [3]:
# Tribe Creation

from eqcorrscan.core.match_filter import Tribe
from obspy.clients.fdsn import Client

client = Client('NCEDC')
catalog = client.get_events(eventid='72572665', includearrivals=True)
# To speed the example we have a catalog of one event, but you can have
# more, we are also only using the first five picks, again to speed the
# example.
catalog[0].picks = catalog[0].picks[0:5]
tribe = Tribe().construct(
    method='from_client', catalog=catalog, client_id='NCEDC', lowcut=2.0,
    highcut=8.0,  samp_rate=20.0, filt_order=4, length=6.0, prepick=0.1,
    swin='all', process_len=3600, all_horiz=True)

Pick for CLV.DPZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for BKS.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for GASB.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for HAST.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for HATC.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!


In [4]:
# Match-filter detection using a Tribe

from obspy import UTCDateTime
party, stream = tribe.client_detect(
    client=client, starttime=UTCDateTime(2016, 1, 2),
    endtime=UTCDateTime(2016, 1, 3), threshold=8, threshold_type='MAD',
    trig_int=6, plotvar=False, return_stream=True)

Last bit of data between 2016-01-02T00:59:20.448391Z and 2016-01-02T01:00:39.548391Z will go unused because it is shorter than a chunk of 3600.0 s
plotvar is depreciated, use plot instead
Last bit of data between 2016-01-02T01:59:20.448391Z and 2016-01-02T02:00:39.548391Z will go unused because it is shorter than a chunk of 3600.0 s
plotvar is depreciated, use plot instead
Last bit of data between 2016-01-02T02:59:20.448391Z and 2016-01-02T03:00:39.548391Z will go unused because it is shorter than a chunk of 3600.0 s
plotvar is depreciated, use plot instead
Last bit of data between 2016-01-02T03:59:20.448391Z and 2016-01-02T04:00:39.548391Z will go unused because it is shorter than a chunk of 3600.0 s
plotvar is depreciated, use plot instead
Last bit of data between 2016-01-02T04:59:20.448391Z and 2016-01-02T05:00:39.548391Z will go unused because it is shorter than a chunk of 3600.0 s
plotvar is depreciated, use plot instead
Last bit of data between 2016-01-02T05:59:20.448391Z and 201

In [5]:
# Generating a Party from a detection csv

detections = read_detections(detection_file) 
party = Party() 
for template in tribe: 
    template_detections = [d for d in detections
                          if d.template_name == template.name]
    family = Family(template=template, detections=template_detections)
    party += family

NameError: name 'read_detections' is not defined

In [ ]:
# Lag-calc using a Party

stream = stream.merge().sort(['station'])
repicked_catalog = party.lag_calc(stream, pre_processed=False,
                                  shift_len=0.2, min_cc=0.4) 

In [ ]:
# Simple example - match-filter.match-filter

from eqcorrscan.core.match_filter import match_filter
from eqcorrscan.utils import pre_processing
from obspy import read

# Read in and process the daylong data
st = read('continuous_data')
# Use the same filtering and sampling parameters as your template!
st = pre_processing.dayproc(
    st, lowcut=2, highcut=10, filt_order=4, samp_rate=50,
    starttime=st[0].stats.starttime.date)
# Read in the templates
templates = []
template_names = ['template_1', 'template_2']
for template_file in template_names:
     templates.append(read(template_file))
detections = match_filter(
     template_names=template_names, template_list=templates, st=st,
     threshold=8, threshold_type='MAD', trig_int=6, plotvar=False, cores=4)

In [ ]:
# To write all detections to one file

for detection in detections:
     detection.write('my_first_detections.csv', append=True)

In [ ]:
# Advanced example

"""
Simple tutorial to demonstrate some of the basic capabilities of the EQcorrscan
matched-filter detection routine.  This builds on the template generation
tutorial and uses those templates.  If you haven't run that tutorial script
then you will need to before you can run this script.
"""

import glob
import logging

from http.client import IncompleteRead
from multiprocessing import cpu_count
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, Stream, read

from eqcorrscan.utils import pre_processing
from eqcorrscan.utils import plotting
from eqcorrscan.core import match_filter

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s")


def run_tutorial(plot=False, process_len=3600, num_cores=cpu_count(),
                 **kwargs):
    """Main function to run the tutorial dataset."""
    # First we want to load our templates
    template_names = glob.glob('tutorial_template_*.ms')

    if len(template_names) == 0:
        raise IOError('Template files not found, have you run the template ' +
                      'creation tutorial?')

    templates = [read(template_name) for template_name in template_names]

    # Work out what stations we have and get the data for them
    stations = []
    for template in templates:
        for tr in template:
            stations.append((tr.stats.station, tr.stats.channel))
    # Get a unique list of stations
    stations = list(set(stations))

    # We will loop through the data chunks at a time, these chunks can be any
    # size, in general we have used 1 day as our standard, but this can be
    # as short as five minutes (for MAD thresholds) or shorter for other
    # threshold metrics. However the chunk size should be the same as your
    # template process_len.

    # You should test different parameters!!!
    start_time = UTCDateTime(2016, 1, 4)
    end_time = UTCDateTime(2016, 1, 5)
    chunks = []
    chunk_start = start_time
    while chunk_start < end_time:
        chunk_end = chunk_start + process_len
        if chunk_end > end_time:
            chunk_end = end_time
        chunks.append((chunk_start, chunk_end))
        chunk_start += process_len

    unique_detections = []

    # Set up a client to access the GeoNet database
    client = Client("GEONET")

    # Note that these chunks do not rely on each other, and could be paralleled
    # on multiple nodes of a distributed cluster, see the SLURM tutorial for
    # an example of this.
    for t1, t2 in chunks:
        # Generate the bulk information to query the GeoNet database
        bulk_info = []
        for station in stations:
            bulk_info.append(('NZ', station[0], '*',
                              station[1][0] + 'H' + station[1][-1], t1, t2))

        # Note this will take a little while.
        print('Downloading seismic data, this may take a while')
        st = Stream()
        for _bulk in bulk_info:
            try:
                st += client.get_waveforms(*_bulk)
            except IncompleteRead:
                print(f"Could not download {_bulk}")
        # Merge the stream, it will be downloaded in chunks
        st.merge()

        # Pre-process the data to set frequency band and sampling rate
        # Note that this is, and MUST BE the same as the parameters used for
        # the template creation.
        print('Processing the seismic data')
        st = pre_processing.shortproc(
            st, lowcut=2.0, highcut=9.0, filt_order=4, samp_rate=20.0,
            num_cores=num_cores, starttime=t1, endtime=t2)
        # Convert from list to stream
        st = Stream(st)

        # Now we can conduct the matched-filter detection
        detections = match_filter.match_filter(
            template_names=template_names, template_list=templates,
            st=st, threshold=8.0, threshold_type='MAD', trig_int=6.0,
            plotvar=plot, plotdir='.', cores=num_cores,
            plot_format='png', **kwargs)

        # Now lets try and work out how many unique events we have just to
        # compare with the GeoNet catalog of 20 events on this day in this
        # sequence
        for master in detections:
            keep = True
            for slave in detections:
                if not master == slave and abs(master.detect_time -
                                               slave.detect_time) <= 1.0:
                    # If the events are within 1s of each other then test which
                    # was the 'best' match, strongest detection
                    if not master.detect_val > slave.detect_val:
                        keep = False
                        print('Removed detection at %s with cccsum %s'
                              % (master.detect_time, master.detect_val))
                        print('Keeping detection at %s with cccsum %s'
                              % (slave.detect_time, slave.detect_val))
                        break
            if keep:
                unique_detections.append(master)
                print('Detection at :' + str(master.detect_time) +
                      ' for template ' + master.template_name +
                      ' with a cross-correlation sum of: ' +
                      str(master.detect_val))
                # We can plot these too
                if plot:
                    stplot = st.copy()
                    template = templates[template_names.index(
                        master.template_name)]
                    lags = sorted([tr.stats.starttime for tr in template])
                    maxlag = lags[-1] - lags[0]
                    stplot.trim(starttime=master.detect_time - 10,
                                endtime=master.detect_time + maxlag + 10)
                    plotting.detection_multiplot(
                        stplot, template, [master.detect_time.datetime])
    print('We made a total of ' + str(len(unique_detections)) + ' detections')
    return unique_detections


if __name__ == '__main__':
    run_tutorial()

In [ ]:
# SLURM example refer to docs 4.2.12.